In [52]:
import requests
import json
import tweepy as tw
import pickle
import collections
from requests_oauthlib import OAuth1
from statistics import mode
from city_to_state import city_to_state_dict

In [6]:
# Store authentication parameters
consumer_key = 'dqFNLbKtUa1K5AohL9yyBML56'
consumer_secret = 'LREzK0HlKlNJFZ0nZr4qktfQaigcGE9KrZpthQGsCFDa5LMTh6'
access_token = '74369840-mi6LxyF0iVzcaIyibzQqUrEAEwEFpFzU2cqzdSLYQ'
access_token_secret = 'q67qlLNZcaf4Hm5rh8NIGhUPAnYQodKBCbM3iqWQYJ5x2'

# Set up API client
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [0]:
# Query API and store results
bernie_results = [result for result in tw.Cursor(api.search, q='Bernie Sanders').items(1000)]
warren_results = [result for result in tw.Cursor(api.search, q='Elizabeth Warren').items(1000)]
biden_results = [result for result in tw.Cursor(api.search, q='Joe Biden').items(1000)]
buttigieg_results = [result for result in tw.Cursor(api.search, q='Pete Buttigieg').items(1000)]
yang_results = [result for result in tw.Cursor(api.search, q='Andrew Yang').items(1000)]

In [0]:
# Given a list of Twitter Status results, returns a list of 2-tuples
# where the first element of the tuple is the tweet text, 
# and the second element is the location
def extract_tweets(results):
  # Filter out any non-status objects
  filtered_results = [result for result in results if isinstance(result, tw.models.Status)]

  tweets = []

  for result in filtered_results:
    tweet = result._json
    tweet_location = tweet['user']['location']
    
    if tweet_location != '':
      tweet_text = tweet['text']
      tweet = (tweet_text, tweet_location)
      tweets.append(tweet)
  
  return tweets

In [0]:
# Extract all tweets from result lists
biden_tweets = extract_tweets(biden_results)
warren_tweets = extract_tweets(warren_results)
bernie_tweets = extract_tweets(bernie_results)
buttigieg_tweets = extract_tweets(buttigieg_results)
yang_tweets = extract_tweets(yang_results)

In [0]:
# Write tweet contents to files
with open('biden_tweets.data', 'wb') as output_file:
  pickle.dump(biden_tweets, output_file)
with open('warren_tweets.data', 'wb') as output_file:
  pickle.dump(warren_tweets, output_file)
with open('bernie_tweets.data', 'wb') as output_file:
  pickle.dump(bernie_tweets, output_file)
with open('buttigieg_tweets.data', 'wb') as output_file:
  pickle.dump(buttigieg_tweets, output_file)
with open('yang_tweets.data', 'wb') as output_file:
  pickle.dump(yang_tweets, output_file)

In [45]:
with open('biden_tweets_cleaned.data', 'rb') as input_file:
    biden_tweets = pickle.load(input_file)
with open('warren_tweets_cleaned.data', 'rb') as input_file:
    warren_tweets = pickle.load(input_file)
with open('bernie_tweets_cleaned.data', 'rb') as input_file:
    bernie_tweets = pickle.load(input_file)
with open('buttigieg_tweets_cleaned.data', 'rb') as input_file:
    buttigieg_tweets = pickle.load(input_file)
with open('yang_tweets_cleaned.data', 'rb') as input_file:
    yang_tweets = pickle.load(input_file)

In [17]:
def text_to_location(loc_query):
    api_key = 'AkTWid9VjTfo9TFzdQV5f5lEdT8PrAY5iM9COWeX1opSpgz6-AzrgBr99kot04CP'
    loc_query = loc_query.replace(' ', '%20')

    url = 'http://dev.virtualearth.net/REST/v1/Locations?query=' + loc_query + '&maxResults=20&key=' + api_key
    r = requests.get(url) 
    x = r.json() 
    y = x['resourceSets']
    state = []

    for i in range(len(y)): 
        d = y[i]['resources']
        for j in d:
            if j['confidence'] == 'High':
                try:
                    state.append(j['address']['adminDistrict'])
                except:
                    return None
    try:
        return mode(state)
    except:
        return None

In [18]:
biden_tweets = [(tweet[0], text_to_location(tweet[1])) for tweet in biden_tweets]
warren_tweets = [(tweet[0], text_to_location(tweet[1])) for tweet in warren_tweets]
bernie_tweets = [(tweet[0], text_to_location(tweet[1])) for tweet in bernie_tweets]
buttigieg_tweets = [(tweet[0], text_to_location(tweet[1])) for tweet in buttigieg_tweets]
yang_tweets = [(tweet[0], text_to_location(tweet[1])) for tweet in yang_tweets]

In [46]:
states = set(["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"])

biden_tweets = [tweet for tweet in biden_tweets if tweet[1] in states]
warren_tweets = [tweet for tweet in warren_tweets if tweet[1] in states]
bernie_tweets = [tweet for tweet in bernie_tweets if tweet[1] in states]
buttigieg_tweets = [tweet for tweet in buttigieg_tweets if tweet[1] in states]
yang_tweets = [tweet for tweet in yang_tweets if tweet[1] in states]

print(len(biden_tweets))
print(len(warren_tweets))
print(len(bernie_tweets))
print(len(buttigieg_tweets))
print(len(yang_tweets))

370
446
475
461
449


In [76]:
all_tweets = biden_tweets + warren_tweets + bernie_tweets + buttigieg_tweets + yang_tweets
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
all_tweet_states = [tweet[1] for tweet in all_tweets if tweet[1] in set(states)]
tweets_by_state = {state: all_tweet_states.count(state)/len(all_tweet_states)*100 for state in states}
tweets_by_state

{'AL': 0.817810086324398,
 'AK': 0.2271694684234439,
 'AZ': 2.3625624716038165,
 'AR': 0.3634711494775102,
 'CA': 15.129486597001362,
 'CO': 1.7264879600181735,
 'CT': 0.8632439800090868,
 'DC': 2.0899591094956835,
 'DE': 0.09086778736937755,
 'FL': 6.769650159018628,
 'GA': 2.1808268968650615,
 'HI': 0.1817355747387551,
 'ID': 0.31803725579282144,
 'IL': 4.407087687414812,
 'IN': 1.4993184915947297,
 'IA': 0.49977283053157656,
 'KS': 0.6815084052703316,
 'KY': 0.9995456610631531,
 'LA': 0.408905043162199,
 'ME': 0.6815084052703316,
 'MD': 1.5447523852794183,
 'MA': 2.453430258973194,
 'MI': 1.8173557473875512,
 'MN': 1.4993184915947297,
 'MS': 0.5452067242162654,
 'MO': 2.58973194002726,
 'MT': 0.3634711494775102,
 'NE': 0.408905043162199,
 'NV': 1.226715129486597,
 'NH': 0.2726033621081327,
 'NJ': 2.271694684234439,
 'NM': 0.7269422989550204,
 'NY': 9.086778736937756,
 'NC': 2.3171285779191275,
 'ND': 0.13630168105406634,
 'OH': 2.7714675147660155,
 'OK': 0.6815084052703316,
 'OR': 2

In [79]:
import plotly.graph_objects as go

tweet_counts = [count for count in tweets_by_state.values()]

fig = go.Figure(data=go.Choropleth(
    locations=states, 
    z = tweet_counts, # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'Blues',
    colorbar_title = "Percentage of all Tweets",
))

fig.update_layout(
    title_text = 'Proportion of Tweets by State',
    geo_scope='usa', # limite map scope to USA
)

fig.show()

In [47]:
# Write tweet contents to files
with open('biden_tweets_cleaned.data', 'wb') as output_file:
  pickle.dump(biden_tweets, output_file)
with open('warren_tweets_cleaned.data', 'wb') as output_file:
  pickle.dump(warren_tweets, output_file)
with open('bernie_tweets_cleaned.data', 'wb') as output_file:
  pickle.dump(bernie_tweets, output_file)
with open('buttigieg_tweets_cleaned.data', 'wb') as output_file:
  pickle.dump(buttigieg_tweets, output_file)
with open('yang_tweets_cleaned.data', 'wb') as output_file:
  pickle.dump(yang_tweets, output_file)